Formerly called minreverse

checkout multioutvar.ipynb

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
from compute import Var, eqvar, Evaluable, ureg, coupled_run, buildidpvars, Impcomp
from inputresolver import reassigneq, eqvars, eqsonly, default_out, mdf_order, getdofs, idx_eqlist, getallvars, invert_edges, var_matched_cons
from inputresolver import reassign
from compute_utils import get_outputs, check_eqs
from representations import bipartite_repr, drawbipartite, digraph_repr
import openmdao.api as om
import sympy as sp
import networkx as nx
from functools import reduce

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In [3]:
class Impcomp(om.ImplicitComponent):
    def initialize(self):
        self.options.declare('equation')
        self.options.declare('output_name')
        
    def setup(self):
        equation = self.options['equation']
        output_name = self.options['output_name']
        self.add_output(output_name)
        original_inputs = [inp for inp in equation.input_names if inp != output_name]
        for name in original_inputs:
            self.add_input(name, val=1.) # add them in the order we lambdify
        self.declare_partials(output_name, equation.input_names)

    def apply_nonlinear(self, inputs, outputs, residuals):
        equation = self.options['equation']
        output_name = self.options['output_name']
        residuals[output_name] = equation.evaldict({**inputs, **outputs})
        
    def linearize(self, inputs, outputs, partials):
        equation = self.options['equation']
        output_name = self.options['output_name']
        J = equation.graddict({**inputs, **outputs})
        for idx, input_name in enumerate(equation.input_names):
            partials[output_name, input_name] = J[idx]

In [71]:
eq = eqlist[0]
eq

(b, c + 1)

In [72]:
ev = Evaluable.fromsympy(eq[0]-eq[1])

In [73]:
ev.input_names

['c', 'b']

In [75]:
prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Impcomp(equation = ev, output_name='b'), promotes=['*'])
model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
model.linear_solver = om.ScipyKrylov()

prob.setup()
prob.set_val('c', 3.)
prob.run_model()

print(prob.get_val('comp.b'))

NL: Newton Converged in 1 iterations
[4.]


In [3]:
a = Var('a')
b = Var('b')
c = Var('c')
d = Var('d')

In [4]:
eqlist = [(b,c+1),(a,b-2),(a,2*d+3)]

In [5]:
eqs = idx_eqlist(eqlist)
eqv = eqvars(eqs)

In [51]:
eqs

{0: (b, c + 1), 1: (a, b - 2), 2: (a, 2*d + 3)}

In [8]:
eqv

{0: {b, c}, 1: {a, b}, 2: {a, d}}

In [9]:
dout = default_out(eqs)

In [10]:
dout

{0: b, 1: a, 2: a}

In [16]:
G, edges_original = bipartite_repr(eqv)
vrs = getallvars(eqs)
eqns = eqs.keys()
edges = invert_edges(edges_original)

In [40]:
not_input = []
n_eqs = len(eqns)

In [45]:
m = gp.Model('minrevassign')
m.setParam('OutputFlag', False )
x = m.addVars(edges, name="assign", vtype=GRB.BINARY)

In [46]:
# Matching eqs:
fcons = m.addConstrs((x.sum(j,'*') == 1 for j in eqns), name='equations')
varcons = m.addConstrs((var_matched_cons(x, j, not_input) for j in vrs), name='variables')
m.setObjective(gp.quicksum([x[(key, var)] for key,var in dout.items()]), GRB.MAXIMIZE)
#m.addConstr(gp.quicksum(x[(r,j)] for r,j in sol)<= n_eqs-1) 

In [47]:
m.optimize()

In [48]:
m.objVal

2.0

In [49]:
sol = [(r,j) for (r, j) in edges if x[r,j].x>1e-6]

In [52]:
sol_outset = {key:val for key,val in sol}

In [53]:
sol_outset

{0: b, 1: a, 2: d}